In [5]:
from Bio import SeqIO
from Bio.Seq import Seq
import os
import statistics as stat

In [6]:
bed_dir_list = ["/home/transposons/Genomes/TAIR10/BedDir/GsBed/", "/home/transposons/Genomes/Oryza_sativa/BedDir/GsBed/", "/home/transposons/Genomes/Glycine_max/BedDir/GsBed/", "/home/transposons/Genomes/Sorghum_bicolor/BedDir/GsBed/"]
fasta_dir_list = ["/home/transposons/Genomes/TAIR10/Fasta/", "/home/transposons/Genomes/Oryza_sativa/Fasta/", "/home/transposons/Genomes/Glycine_max/Fasta/", "/home/transposons/Genomes/Sorghum_bicolor/Fasta/"]
for path in bed_dir_list:
    assert os.path.exists(path)
for path in fasta_dir_list:
    assert os.path.exists(path)


In [7]:
bed_list = [[bed_dir + "/" + name for name in os.listdir(bed_dir)] for bed_dir in bed_dir_list]
fasta_list = [[fasta_dir + "/" + name for name in os.listdir(fasta_dir)] for fasta_dir in fasta_dir_list]
assert len(bed_list) == len(fasta_list)

In [8]:
# unpack every inner list into a singular list
bed_list = [item for sublist in bed_list for item in sublist]
fasta_list = [item for sublist in fasta_list for item in sublist]

In [9]:
def sort_key(file_name):
    data = file_name.split('_')  
    genome_name, chr_name = '_'.join(data[:-1]), data[-1].split('.')[0] # Extract the genome and chromosome names from the file name
    return (genome_name, int(chr_name.replace('chr', '')))  # Sort by genome name, then chromosome number

In [10]:
bed_list.sort(key=sort_key)
fasta_list.sort(key=sort_key)

In [11]:
for i in range(len(bed_list)):
    print(os.path.basename(bed_list[i]), os.path.basename(fasta_list[i]))

Glycine_max_chr1.bed Glycine_max_chr1.fa
Glycine_max_chr2.bed Glycine_max_chr2.fa
Glycine_max_chr3.bed Glycine_max_chr3.fa
Glycine_max_chr4.bed Glycine_max_chr4.fa
Glycine_max_chr5.bed Glycine_max_chr5.fa
Glycine_max_chr6.bed Glycine_max_chr6.fa
Glycine_max_chr7.bed Glycine_max_chr7.fa
Glycine_max_chr8.bed Glycine_max_chr8.fa
Glycine_max_chr9.bed Glycine_max_chr9.fa
Glycine_max_chr10.bed Glycine_max_chr10.fa
Glycine_max_chr11.bed Glycine_max_chr11.fa
Glycine_max_chr12.bed Glycine_max_chr12.fa
Glycine_max_chr13.bed Glycine_max_chr13.fa
Glycine_max_chr14.bed Glycine_max_chr14.fa
Glycine_max_chr15.bed Glycine_max_chr15.fa
Glycine_max_chr16.bed Glycine_max_chr16.fa
Glycine_max_chr17.bed Glycine_max_chr17.fa
Glycine_max_chr18.bed Glycine_max_chr18.fa
Glycine_max_chr19.bed Glycine_max_chr19.fa
Glycine_max_chr20.bed Glycine_max_chr20.fa
Oryza_sativa_chr1.bed Oryza_sativa_chr1.fa
Oryza_sativa_chr2.bed Oryza_sativa_chr2.fa
Oryza_sativa_chr3.bed Oryza_sativa_chr3.fa
Oryza_sativa_chr4.bed Oryza_s

In [12]:
def get_alignment(seq1, seq2, match, mismatch, gap_open, gap_continue):
    score_matrix = [[0 for x in range(len(seq2) + 1)] for y in range(len(seq1) + 1)]
    del_gap_matrix = [[0 for x in range(len(seq2) + 1)] for y in range(len(seq1) + 1)]
    ins_gap_matrix = [[0 for x in range(len(seq2) + 1)] for y in range(len(seq1) + 1)]

    max_score = 0
    max_i = 0
    max_j = 0

    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            m = match if seq1[i - 1] == seq2[j - 1] else mismatch
            m += score_matrix[i - 1][j - 1]


            del_gap_matrix[i][j] = max(score_matrix[i - 1][j] + gap_open, del_gap_matrix[i - 1][j] + gap_continue)
            ins_gap_matrix[i][j] = max(score_matrix[i][j-1] + gap_open, ins_gap_matrix[i][j - 1] + gap_continue)
            score_matrix[i][j] = max(m, del_gap_matrix[i][j], ins_gap_matrix[i][j], 0)

            if score_matrix[i][j] > max_score:
                max_score = score_matrix[i][j]
                max_i = i
                max_j = j

    i = max_i
    j = max_j
    loc_align = 0
    sim_counter = 0

    while score_matrix[i][j] > 0:
        s = match if seq1[i - 1] == seq2[j - 1] else mismatch
        if score_matrix[i][j] == score_matrix[i - 1][j - 1] + s:
            if s == match:
                sim_counter += 1
            i -= 1
            j -=1

        elif score_matrix[i][j] == del_gap_matrix[i][j]:
            i -= 1
        else:
            j -=1

        loc_align += 1
    
    return loc_align, sim_counter


In [13]:
get_alignment("CCCCAATAAGGG", "AAAAAAAAAA", 2, -3, -5, -2)

(5, 4)

In [14]:
def read_bed(path):
    l = []
    with open(path, 'r') as file:
        for line in file.readlines()[1:]:
            data = line.split()
            l.append([int(x) for x in data[3:7]])

    return l

In [27]:
max_ppt_distance = 400 
min_ppt_size = 12

In [28]:
def find_ppt(seq, replace_char, with_char):
    replace_seq = seq.replace(replace_char, with_char)
    seq_all = ''.join([with_char for _ in range(100)])
    r = get_alignment(replace_seq, seq_all, 2, -3, -5, -2)

    return r

In [29]:
from concurrent.futures import ProcessPoolExecutor

def process_ppt(fasta_path, bed_path):
    rc_counter = 0
    found_counter = 0
    length_list = []
    print(os.path.basename(bed_path))
    seq = str(SeqIO.read(fasta_path, 'fasta').seq)
    for ele in read_bed(bed_path):
        inner = seq[ele[1]:ele[2]]

        inner_size = len(inner)
        if inner_size > max_ppt_distance * 2:
            left_end = max_ppt_distance
            right_start = inner_size - max_ppt_distance
        else:
            half = int(inner_size / 2)
            left_end = half
            right_start = inner_size - half

        left_seq = inner[:left_end]
        right_seq = inner[right_start:]

        left_align = find_ppt(left_seq, 'C', 'T')
        right_align = find_ppt(right_seq, 'G', 'A')

        is_left = left_align[0] >= min_ppt_size
        is_right = right_align[0] >= min_ppt_size

        if is_left and not is_right:
            rc_counter += 1

        if is_right or is_left:
            found_counter += 1

        length_list.append(max(left_align[0], right_align[0]))

    return rc_counter, found_counter, length_list

In [30]:
with ProcessPoolExecutor() as executor:
    results = list(executor.map(process_ppt, fasta_list, bed_list))

rc_counter = 0
found_counter = 0
length_list = []
for result in results:
    rc_counter += result[0]
    found_counter += result[1]
    length_list.extend(result[2])

Glycine_max_chr4.bedGlycine_max_chr5.bedGlycine_max_chr8.bedGlycine_max_chr1.bedGlycine_max_chr7.bedGlycine_max_chr2.bedGlycine_max_chr9.bedGlycine_max_chr3.bedGlycine_max_chr11.bedGlycine_max_chr12.bedGlycine_max_chr6.bedGlycine_max_chr10.bedGlycine_max_chr15.bed

Glycine_max_chr13.bedGlycine_max_chr16.bed










Glycine_max_chr14.bed


Glycine_max_chr17.bed
Glycine_max_chr18.bed
Glycine_max_chr19.bed
Glycine_max_chr20.bed
Oryza_sativa_chr1.bed
Oryza_sativa_chr2.bed
Oryza_sativa_chr3.bed
Oryza_sativa_chr4.bed
Oryza_sativa_chr5.bed
Oryza_sativa_chr6.bed
Oryza_sativa_chr7.bed
Oryza_sativa_chr8.bed
Oryza_sativa_chr9.bed
Oryza_sativa_chr10.bed
Oryza_sativa_chr11.bed
Oryza_sativa_chr12.bed
Sorghum_bicolor_chr1.bed
Sorghum_bicolor_chr2.bed
Sorghum_bicolor_chr3.bed
Sorghum_bicolor_chr4.bed
Sorghum_bicolor_chr5.bed
Sorghum_bicolor_chr6.bed
Sorghum_bicolor_chr7.bed
Sorghum_bicolor_chr8.bed
Sorghum_bicolor_chr9.bed
Sorghum_bicolor_chr10.bed
TAIR10_chr1.bed
TAIR10_chr2.bed
TAIR10_chr3.bed
TAI

In [31]:
rc_counter

735

In [32]:
print("Mean:", stat.mean(length_list))
print("Median", stat.median(length_list))
print("STD:", stat.stdev(length_list))
print("Max:", max(length_list))
print("Min:", min(length_list))

Mean: 59.26119729080184
Median 53.0
STD: 28.43045316633688
Max: 129
Min: 7


In [33]:
found_counter

9133

In [35]:
total = 0
for bed_path in bed_list:
    total += len(read_bed(bed_path))
print(total)

9154


Out of 9154

With 400 and 10: Found 9139

With 400 and 11: Found 9139

With 400 and 12: Found 9133

With 400 and 13: Found 9103

With 400 and 15: Found 8783

With 400 and 20: Found 8645